<a href="https://colab.research.google.com/github/joekelly211/masfi/blob/main/6_scenarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports, directories and global functions

In [ ]:
# Define base directory
# Use '/content/drive/MyDrive/' for a personal drive
# Use '/gdrive/Shareddrives/' for a shared drive (must be created first)

base_dir = "/gdrive/Shareddrives/masfi"
# base_dir = '/content/drive/MyDrive/masfi'

# Mount Google Drive
from google.colab import drive
import os
import sys
if base_dir.startswith('/gdrive/Shareddrives/'):
  drive.mount('/gdrive', force_remount=True)
elif base_dir.startswith('/content/drive/MyDrive/'):
  drive.mount('/content/drive', force_remount=True)
  os.makedirs(base_dir, exist_ok=True)
else: print("Create a base_dir beginning with '/gdrive/Shareddrives/' or '/content/drive/MyDrive/'.")

_path_to_add = os.path.realpath(base_dir)
if _path_to_add not in sys.path:
    sys.path.append(_path_to_add)

In [ ]:
# Capture outputs
%%capture
# Installs and upgrades
!pip install astropy
!pip install geopandas
!pip install rasterio
!apt-get install -y gdal-bin
!pip install xgboost

In [ ]:
# Reload imports, replacing those in the cache
%reload_ext autoreload
%autoreload 2
# Imports
from astropy.convolution import convolve, Gaussian2DKernel
from contextlib import contextmanager
try: import cupy # Only works on GPU runtime
except: None
import gc
import geopandas as gpd
from google.colab import runtime
import json
import math
from os import makedirs
from os.path import join, exists
from osgeo import gdal, ogr
gdal.UseExceptions()
import ipywidgets as widgets
import numpy as np
import pandas as pd
from pathlib import Path
import psutil
import rasterio
from rasterio.features import rasterize
import re
from scipy import ndimage
import shutil
from shutil import copyfile, rmtree
import subprocess
import time
import xgboost as xgb
import warnings

In [ ]:
# Define directories
areas_dir = join(base_dir, "1_areas")
polygons_dir = join(areas_dir, "polygons")

feature_dir = join(base_dir, "3_features")
feature_resampled_dir = join(feature_dir, "resampled")
feature_binary_dir = join(feature_dir, "binary")
feature_final_dir = join(feature_dir, "final")
models_dir = join(base_dir, "5_models")
scenarios_dir = join(base_dir, "6_scenarios")
masks_dir = join(scenarios_dir, "scenario_masks")

# Create directories
makedirs(scenarios_dir, exist_ok=True)
makedirs(masks_dir, exist_ok=True)

In [ ]:
# Global function: export an array as a .tif
template_tif_path = join(areas_dir, "template.tif")
nodatavalue = -1111111
compress = True
def export_array_as_tif(input_array, output_tif, template=template_tif_path, nodatavalue=nodatavalue, compress=compress, dtype=gdal.GDT_Float32):
    template_ds = gdal.Open(template)
    template_band = template_ds.GetRasterBand(1)
    template_dimensions, template_projection = template_ds.GetGeoTransform(), template_ds.GetProjection()
    if compress: options = ["COMPRESS=DEFLATE", "PREDICTOR=3", "ZLEVEL=9"]
    else: options = []
    driver = gdal.GetDriverByName("GTiff").Create(output_tif, template_band.XSize, template_band.YSize, 1, dtype, options=options)
    driver.GetRasterBand(1).WriteArray(input_array)
    driver.GetRasterBand(1).SetNoDataValue(nodatavalue)
    driver.SetGeoTransform(template_dimensions)
    driver.SetProjection(template_projection)
    template_ds = driver = None

# Global function: burn a polygon to raster
def burn_polygon_to_raster(raster_path, polygon_path, fixed=True, fixed_value=1, column_name=None, all_touched=True):
    raster = vector = None
    try:
        raster = gdal.Open(raster_path, gdal.GA_Update)
        vector = ogr.Open(polygon_path)
        if not raster or not vector:
            raise ValueError("Cannot open input files")
        layer = vector.GetLayer()
        options = ["ALL_TOUCHED=TRUE"] if all_touched else []
        if fixed:
            gdal.RasterizeLayer(raster, [1], layer, burn_values=[fixed_value], options=options)
        else:
            attr_name = column_name or layer.GetLayerDefn().GetFieldDefn(0).GetName()
            options.append(f"ATTRIBUTE={attr_name}")
            gdal.RasterizeLayer(raster, [1], layer, options=options)
    finally:
        if raster: raster.FlushCache()
        raster = vector = None

def edge_effects(binary_array):
  # Set smoothing kernel and precision
  kernel, precision = Gaussian2DKernel(x_stddev=3, y_stddev=3), 2
  # Reclassify for binary differentiation after proximity conversion
  differentiator_array = binary_array.copy()
  differentiator_array[differentiator_array == 1] = 10
  # Positive proximity
  positive_distances = ndimage.distance_transform_edt(binary_array == 0) # target pixels
  positive_proximity_array = np.where(positive_distances > 2, 0, positive_distances) # max distance 2
  # Negative proximity
  negative_distances = ndimage.distance_transform_edt(binary_array == 1) # target pixels
  negative_proximity_array = np.where(negative_distances > 2, 0, negative_distances) # max distance 2
  # Sum proximities and differentiator
  pixel_prox_summed =  differentiator_array + positive_proximity_array + negative_proximity_array
  # Reclassify for better semantic understanding of pixel proximity
  pixel_prox_reclassed = pixel_prox_summed.copy()
  pixel_prox_reclass_table = [(0, 0, -4), (1, 1, -1), (1.4, 1.5, -2), (2, 2, -3), (10, 10, 3), (11, 11, 0), (11.4, 11.5, 1), (12, 12, 2)]
  for min_value, max_value, new_value in pixel_prox_reclass_table:
    pixel_prox_reclassed[(pixel_prox_reclassed >= min_value) & (pixel_prox_reclassed <= max_value)] = new_value
  # Smooth binary array using 2D convolution
  binary_smoothed = convolve(binary_array, kernel, boundary='extend')
  # Sum pixel proximity and smoothed binary array
  edge_effects_array = np.round(pixel_prox_reclassed + binary_smoothed, precision)
  return edge_effects_array

# Select model

In [ ]:
# Select a model
model_exists = False
for subdir, dirs, files in os.walk(models_dir):
  for file in files:
    if file == 'model.json':
      print(f'selected_model = "{subdir.split(f"{models_dir}/",1)[1]}"')
      model_exists = True
if not model_exists:
  print("No model exists.")

In [ ]:
selected_model = "agbd_tekai_250625_003858"

# Define model directories
selected_model_dir = join(models_dir,selected_model)
selected_model_json = join(selected_model_dir, "model.json")
selected_model_descr_dir = join(selected_model_dir, "model_description.json")
selected_model_dataset_path = join(selected_model_dir, f"{selected_model}.pkl")
selected_model_dataset = pd.read_pickle(selected_model_dataset_path)

# Read description for model dataset attributes
with open(join(selected_model_dir,"model_dataset_description.json")) as model_dataset_description_json:
  model_dataset_description = json.load(model_dataset_description_json)
model_dataset_name = model_dataset_description["model_dataset_name"]
number_of_columns = model_dataset_description["number_of_columns"]
number_of_rows = model_dataset_description["number_of_rows"]
id_column = model_dataset_description["id_column"]
selected_target = model_dataset_description["selected_target"]
uncertainty = model_dataset_description["uncertainty"]
covariates_renamed = model_dataset_description["covariates_renamed"]
covariates_categorised = model_dataset_description["covariates_categorised"]
selected_features = model_dataset_description["selected_features"] + model_dataset_description["covariates_renamed"]
categorical_columns = model_dataset_description["categorical_columns"]
descriptive_parameters = model_dataset_description["descriptive_parameters"]
filter_parameter = model_dataset_description["filter_parameter"]
filter_values_to_include = model_dataset_description["filter_values_to_include"]
sample_imported_dataset = model_dataset_description["sample_imported_dataset"]
sample_imported_dataset_by_percent = model_dataset_description["sample_imported_dataset_by_percent"]
sample_imported_dataset_value = model_dataset_description["sample_imported_dataset_value"]

covariates = [covariate[4:] for covariate in covariates_renamed]

# Create scenarios model directory
scenarios_model_dir = join(scenarios_dir, selected_model)
makedirs(scenarios_model_dir, exist_ok=True)

# Copy model_dataset_description.json
with open(join(scenarios_model_dir, "model_dataset_description.json"), "w") as file:
  file.write(json.dumps(model_dataset_description))

# Create subdirectories
features_dir = join(scenarios_model_dir, "features")
tile_templates_dir = join(scenarios_model_dir, 'tile_templates')
tile_features_dir = join(scenarios_model_dir, "tile_features")
tile_feature_stacks_dir = join(scenarios_model_dir, "tile_feature_stacks")
tile_prediction_cache_dir = join(scenarios_model_dir,"tile_prediction_cache")
scenario_predictions_unmasked_dir = join(scenarios_model_dir,"scenario_predictions_unmasked")
scenario_predictions_dir = join(scenarios_model_dir, "scenario_predictions")

makedirs(features_dir, exist_ok=True)
makedirs(tile_templates_dir, exist_ok=True)
makedirs(tile_features_dir, exist_ok=True)
makedirs(tile_feature_stacks_dir, exist_ok=True)
makedirs(tile_prediction_cache_dir, exist_ok=True)
makedirs(scenario_predictions_unmasked_dir, exist_ok=True)
makedirs(scenario_predictions_dir, exist_ok=True)

# Copy features from the final features directory
for feature in os.listdir(feature_final_dir):
  if feature not in os.listdir(features_dir):
    feature_original_path = join(feature_final_dir, feature)
    feature_copy_path = join(features_dir, feature)
    copyfile(feature_original_path, feature_copy_path)
print(f"All features present in the following directory have already been copied over: {feature_final_dir}")

# Define yearly scenarios

In [ ]:
# Yearly scenarios may include years after the model scenario if feature data is available later than GEDI data.
# Date of the prediction is ~December 31st, e.g. '2024' is 31/12/2024, requiring features up to 2024.

model_scenario_override = None # set if cannot be automatically determined from model features

yearly_features = ["forest_with_edge_effects", "disturbance_with_edge_effects"]

# Remove the 'fea_' prefix from each feature
model_features = sorted([feature[4:] for feature in selected_features])

# Create a list of feature years from the model's features
model_feature_years = []
for feature in model_features:
  for yearly_feature in yearly_features:
    if yearly_feature in feature:
      model_feature_years.append(int(feature[-4:]))

# Determine the model scenario from the maximum year
if model_scenario_override != None: model_scenario = model_scenario_override
else: model_scenario = max(model_feature_years)
model_scenario_filename = f"{model_scenario}.csv"
model_scenario_dir = join(scenarios_model_dir, model_scenario_filename)
print(f"The maximum year used in the model is {model_scenario}, which has been created as the first scenario.\n")
print(f"The {model_scenario} scenario feature list has been saved to:\n {model_scenario_dir}\n")
print(f"Ensure all features in this list have been copied to:\n{features_dir}\n")

# Save the model scenario features as a .csv
pd.DataFrame(model_features).to_csv(model_scenario_dir, index=False)

# Determine available feature years
final_feature_years = []
for final_feature in os.listdir(feature_final_dir):
  if final_feature.endswith('.tif') and final_feature[-9] == '_':
    try: final_feature_years.append(int(final_feature[-8:-4]))
    except: continue

# Find the first and last feature years
first_feature_year = min(final_feature_years)
last_feature_year = max(final_feature_years)
additional_feature_years = last_feature_year - model_scenario
print(f"The first available feature year is {first_feature_year} and the last is {last_feature_year}.\n")

# Calculate the range of scenario years and minimum scenario year
model_scenario_year_range = max(model_feature_years) - (min(model_feature_years))
minimum_yearly_scenario = first_feature_year + model_scenario_year_range
print(f"The earliest scenario year that can be predicted is {minimum_yearly_scenario}.")
print(f"The latest scenario year that can be predicted is {last_feature_year}.")
print(f"This is based on the number of yearly features used to train the model and the total availability of features.")

In [ ]:
# Select constant features which are the same in every scenario, e.g. topography
print("constant_features = [")
for feature in model_features:
  if "beam" not in feature and "sensitivity" not in feature:
    print(f'  "{feature}",')
print("]")

In [ ]:
constant_features = [
  "coast_proximity_km",
  "latitude",
  "longitude",
  "topo_dtm_smooth_aspect_cosine",
  "topo_dtm_smooth_aspect_sine",
  "topo_dtm_smooth_circular_variance_aspect_03",
  "topo_dtm_smooth_circular_variance_aspect_07",
  "topo_dtm_smooth_circular_variance_aspect_11",
  "topo_dtm_smooth_deviation_mean_elevation_03",
  "topo_dtm_smooth_deviation_mean_elevation_07",
  "topo_dtm_smooth_deviation_mean_elevation_11",
  "topo_dtm_smooth_eastness",
  "topo_dtm_smooth_elevation",
  "topo_dtm_smooth_northness",
  "topo_dtm_smooth_profile_curvature",
  "topo_dtm_smooth_roughness_03",
  "topo_dtm_smooth_roughness_07",
  "topo_dtm_smooth_roughness_11",
  "topo_dtm_smooth_slope",
  "topo_dtm_smooth_stream_power_index_log10",
  "topo_dtm_smooth_surface_area_ratio",
  "topo_dtm_smooth_tangential_curvature",
  "topo_dtm_smooth_topographic_position_index_03",
  "topo_dtm_smooth_topographic_position_index_07",
  "topo_dtm_smooth_topographic_position_index_11",
  "topo_dtm_smooth_topographic_ruggedness_index",
  "topo_dtm_smooth_topographic_wetness_index",
  "topo_dtm_unsmooth_aspect_cosine",
  "topo_dtm_unsmooth_aspect_sine",
  "topo_dtm_unsmooth_circular_variance_aspect_03",
  "topo_dtm_unsmooth_circular_variance_aspect_07",
  "topo_dtm_unsmooth_circular_variance_aspect_11",
  "topo_dtm_unsmooth_deviation_mean_elevation_03",
  "topo_dtm_unsmooth_deviation_mean_elevation_07",
  "topo_dtm_unsmooth_deviation_mean_elevation_11",
  "topo_dtm_unsmooth_eastness",
  "topo_dtm_unsmooth_elevation",
  "topo_dtm_unsmooth_northness",
  "topo_dtm_unsmooth_profile_curvature",
  "topo_dtm_unsmooth_roughness_03",
  "topo_dtm_unsmooth_roughness_07",
  "topo_dtm_unsmooth_roughness_11",
  "topo_dtm_unsmooth_slope",
  "topo_dtm_unsmooth_stream_power_index_log10",
  "topo_dtm_unsmooth_surface_area_ratio",
  "topo_dtm_unsmooth_tangential_curvature",
  "topo_dtm_unsmooth_topographic_position_index_03",
  "topo_dtm_unsmooth_topographic_position_index_07",
  "topo_dtm_unsmooth_topographic_position_index_11",
  "topo_dtm_unsmooth_topographic_ruggedness_index",
  "topo_dtm_unsmooth_topographic_wetness_index",
]

In [ ]:
# Note that areas that were forested in a historic yearly scenario but were reservoirs
# or flooded at the time topographic feature data was collected (~2014 for GLO-30 DEM)
# will be predicted based on a flat topography at the elevation of the water's surface.

# Set scenario features as all non-constant features
scenario_features = sorted(list(set(model_features) - set(constant_features)))

# Create feature lists for all possible yearly scenarios
for yearly_scenario in range(minimum_yearly_scenario, last_feature_year +1):
  year_difference = model_scenario - yearly_scenario
  yearly_scenario_features = []
  for scenario_feature in scenario_features:
    try:
      year_change = int(scenario_feature[-4:]) - year_difference
      yearly_scenario_feature = scenario_feature[:-4] + str(year_change)
      yearly_scenario_features.append(yearly_scenario_feature)
    except: yearly_scenario_features.append(scenario_feature)
  # Compile yearly features and save as a .csv
  yearly_scenario_features = sorted(yearly_scenario_features + constant_features)
  yearly_scenario_filename = f"{yearly_scenario}.csv"
  yearly_scenario_dir = join(scenarios_model_dir,yearly_scenario_filename)
  pd.DataFrame(yearly_scenario_features).to_csv(yearly_scenario_dir, index=False)

# Open the most recent yearly scenario feature list
most_recent_scenario_csv = join(scenarios_model_dir,f"{last_feature_year}.csv")
most_recent_scenario_features = pd.Series.tolist(pd.read_csv(most_recent_scenario_csv).iloc[:,0])

print(f"Lists of features for all possible yearly scenarios have been exported to {scenarios_model_dir}/.")
print(f"Ensure all features in these lists have been copied to:\n{features_dir}\n")

# Create a 'no disturbance' feature for alternate scenarios.
# Assumes the minimum possible value is present in the first scenario year.
minimum_disturbance_name = f"disturbance_with_edge_effects_0000"
minimum_disturbance_path = join(features_dir, f"{minimum_disturbance_name}.tif")
if not exists(minimum_disturbance_path):
  example_disturbance = join(features_dir, f"disturbance_with_edge_effects_{first_feature_year}.tif")
  example_disturbance_array = gdal.Open(example_disturbance).ReadAsArray()
  minimum_disturbance_value = example_disturbance_array.min()
  minimum_disturbance_array = np.where(example_disturbance_array, minimum_disturbance_value, minimum_disturbance_value)
  export_array_as_tif(minimum_disturbance_array, minimum_disturbance_path, template = example_disturbance)
  print(f"The minimum disturbance value is {minimum_disturbance_value}\n,")
  print(f"which has been used to create the 'minimum disturbance' feature {minimum_disturbance_name}.")
else: print(f"The minimum disturbance feature {minimum_disturbance_name} already exists.")

# Define alternate scenarios (optional)

## Disturbance requirement tool

In [ ]:
# Tool to identify required scenarios for forest AGBD disturbance mapping
use_tool = True

def select_forest_scenarios():
    # Initialize variables
    calculation_note = None
    is_specific_effects_calculation = False

    # Print header
    print("\n" + "="*50)
    print("\nFOREST DISTURBANCE SELECTOR\n")
    print("="*50 + "\n")

    # Step 1: Collect disturbance type
    disturbance_prompt = (
        "Select disturbance type:\n"
        "1. Degradation\n"
        "2. Deforestation\n"
        "3. Disturbance (degradation + deforestation)\n\n"
        "Enter your choice (1-3): "
    )
    disturbance_type = input(disturbance_prompt)

    # Step 2: Year of interest
    print("\n")
    year_of_interest = input("Enter year of interest: ")
    print("\n")

    # Step 3: Collect baseline type
    baseline_prompt = (
        "Select baseline type:\n"
        "1. Since oldgrowth state\n"
        "2. Since a baseline year\n"
        "3. Effect before first available disturbance year\n"
        "4. Effect of a specific year\n"
        "5. Effect in the same year\n\n"
        "Enter your choice (1-5): "
    )
    baseline_type = input(baseline_prompt)

    # Initialize result variables
    selected_difference = None
    scenario_pair = None
    other_requirements = []
    is_specific_effects_calculation = (baseline_type in ["3", "4", "5"])

    # Process based on baseline type
    if baseline_type == "1":  # Since oldgrowth state
        if disturbance_type == "1":  # Degradation
            selected_difference = f"{year_of_interest}_degradation_since_oldgrowth"
            scenario_pair = (year_of_interest, f"{year_of_interest}_oldgrowth")

        elif disturbance_type == "2":  # Deforestation
            selected_difference = f"{year_of_interest}_deforestation_since_oldgrowth"
            # This requires both degradation_since_oldgrowth and disturbance_since_oldgrowth
            deg_oldgrowth_diff = f"{year_of_interest}_degradation_since_oldgrowth"
            deg_oldgrowth_pair = (year_of_interest, f"{year_of_interest}_oldgrowth")
            dist_oldgrowth_diff = f"{year_of_interest}_disturbance_since_oldgrowth"
            dist_oldgrowth_pair = (year_of_interest, f"{year_of_interest}_oldgrowth_all_land")

            other_requirements.append((deg_oldgrowth_diff, deg_oldgrowth_pair))
            other_requirements.append((dist_oldgrowth_diff, dist_oldgrowth_pair))
            calculation_note = f"Calculated as {dist_oldgrowth_diff} - {deg_oldgrowth_diff}"

        elif disturbance_type == "3":  # Total disturbance (degradation + deforestation)
            selected_difference = f"{year_of_interest}_disturbance_since_oldgrowth"
            scenario_pair = (year_of_interest, f"{year_of_interest}_oldgrowth_all_land")

    elif baseline_type == "2":  # Since a baseline year
        # Get and validate baseline year
        print("\n")
        baseline_year = input("Enter baseline year (must be before year of interest): ")
        if int(baseline_year) >= int(year_of_interest):
            print("\nError: Baseline year must be before year of interest")
            return None

        baseline_year_plus1 = str(int(baseline_year) + 1)

        if disturbance_type == "1":  # Degradation
            selected_difference = f"{year_of_interest}_degradation_since_{baseline_year_plus1}"
            scenario_pair = (year_of_interest, f"{year_of_interest}_no_degradation_since_{baseline_year_plus1}")

        elif disturbance_type == "2":  # Deforestation
            selected_difference = f"{year_of_interest}_deforestation_since_{baseline_year_plus1}"
            # This requires both degradation_since and disturbance_since for the same period
            deg_since_diff = f"{year_of_interest}_degradation_since_{baseline_year_plus1}"
            deg_since_pair = (year_of_interest, f"{year_of_interest}_no_degradation_since_{baseline_year_plus1}")
            dist_since_diff = f"{year_of_interest}_disturbance_since_{baseline_year_plus1}"
            dist_since_pair = (year_of_interest, f"{year_of_interest}_no_disturbance_since_{baseline_year_plus1}")

            other_requirements.append((deg_since_diff, deg_since_pair))
            other_requirements.append((dist_since_diff, dist_since_pair))
            calculation_note = f"Calculated as {dist_since_diff} - {deg_since_diff}"

        elif disturbance_type == "3":  # Total disturbance since baseline
            selected_difference = f"{year_of_interest}_disturbance_since_{baseline_year_plus1}"
            scenario_pair = (year_of_interest, f"{year_of_interest}_no_disturbance_since_{baseline_year_plus1}")

    elif baseline_type == "3":  # Effect before first available disturbance year
        # Get and validate first available disturbance year
        print("\n")
        first_year = input("Enter first available disturbance year in your data: ")
        if int(first_year) >= int(year_of_interest):
            print("\nError: first available disturbance year must be before year of interest")
            return None

        if disturbance_type == "1":  # Degradation
            selected_difference = f"{year_of_interest}_effect_of_degradation_before_{first_year}"

            # Define required difference components
            since_first = f"{year_of_interest}_degradation_since_{first_year}"
            since_first_pair = (year_of_interest, f"{year_of_interest}_no_degradation_since_{first_year}")

            oldgrowth_difference = f"{year_of_interest}_degradation_since_oldgrowth"
            oldgrowth_pair = (year_of_interest, f"{year_of_interest}_oldgrowth")

            other_requirements.append((since_first, since_first_pair))
            other_requirements.append((oldgrowth_difference, oldgrowth_pair))
            calculation_note = f"Calculated as {oldgrowth_difference} - {since_first}"

        elif disturbance_type == "2":  # Deforestation
            selected_difference = f"{year_of_interest}_effect_of_deforestation_before_{first_year}"

            # Define required difference components for deforestation_since_first
            deg_since_first = f"{year_of_interest}_degradation_since_{first_year}"
            deg_since_first_pair = (year_of_interest, f"{year_of_interest}_no_degradation_since_{first_year}")
            dist_since_first = f"{year_of_interest}_disturbance_since_{first_year}"
            dist_since_first_pair = (year_of_interest, f"{year_of_interest}_no_disturbance_since_{first_year}")

            # Define required difference components for deforestation_since_oldgrowth
            deg_oldgrowth_diff = f"{year_of_interest}_degradation_since_oldgrowth"
            deg_oldgrowth_pair = (year_of_interest, f"{year_of_interest}_oldgrowth")
            dist_oldgrowth_diff = f"{year_of_interest}_disturbance_since_oldgrowth"
            dist_oldgrowth_pair = (year_of_interest, f"{year_of_interest}_oldgrowth_all_land")

            other_requirements.append((deg_since_first, deg_since_first_pair))
            other_requirements.append((dist_since_first, dist_since_first_pair))
            other_requirements.append((deg_oldgrowth_diff, deg_oldgrowth_pair))
            other_requirements.append((dist_oldgrowth_diff, dist_oldgrowth_pair))

            defor_since_first = f"{year_of_interest}_deforestation_since_{first_year}"
            defor_since_oldgrowth = f"{year_of_interest}_deforestation_since_oldgrowth"
            calculation_note = f"Calculated as {defor_since_oldgrowth} - {defor_since_first}"

        elif disturbance_type == "3":  # Total disturbance before first year
            selected_difference = f"{year_of_interest}_effect_of_disturbance_before_{first_year}"

            # Define required difference components
            since_first = f"{year_of_interest}_disturbance_since_{first_year}"
            since_first_pair = (year_of_interest, f"{year_of_interest}_no_disturbance_since_{first_year}")

            oldgrowth_difference = f"{year_of_interest}_disturbance_since_oldgrowth"
            oldgrowth_pair = (year_of_interest, f"{year_of_interest}_oldgrowth_all_land")

            other_requirements.append((since_first, since_first_pair))
            other_requirements.append((oldgrowth_difference, oldgrowth_pair))
            calculation_note = f"Calculated as {oldgrowth_difference} - {since_first}"

    elif baseline_type == "4":  # Effect of a specific year
        # Get and validate specific year
        print("\n")
        specific_year = input("Enter the specific year whose effect you want to measure: ")
        if int(specific_year) >= int(year_of_interest):
            print("\nError: Specific year must be before year of interest")
            return None

        specific_year_plus1 = str(int(specific_year) + 1)

        if disturbance_type == "1":  # Degradation
            selected_difference = f"{year_of_interest}_effect_of_degradation_in_{specific_year}"

            # Define required difference components
            since_specific = f"{year_of_interest}_degradation_since_{specific_year}"
            since_specific_pair = (year_of_interest, f"{year_of_interest}_no_degradation_since_{specific_year}")

            since_after = f"{year_of_interest}_degradation_since_{specific_year_plus1}"
            since_after_pair = (year_of_interest, f"{year_of_interest}_no_degradation_since_{specific_year_plus1}")

            other_requirements.append((since_specific, since_specific_pair))
            other_requirements.append((since_after, since_after_pair))
            calculation_note = f"Calculated as {since_specific} - {since_after}"

        elif disturbance_type == "2":  # Deforestation
            selected_difference = f"{year_of_interest}_effect_of_deforestation_in_{specific_year}"

            # Define components for since_specific
            deg_since_specific = f"{year_of_interest}_degradation_since_{specific_year}"
            deg_since_specific_pair = (year_of_interest, f"{year_of_interest}_no_degradation_since_{specific_year}")
            dist_since_specific = f"{year_of_interest}_disturbance_since_{specific_year}"
            dist_since_specific_pair = (year_of_interest, f"{year_of_interest}_no_disturbance_since_{specific_year}")

            # Define components for since_after
            deg_since_after = f"{year_of_interest}_degradation_since_{specific_year_plus1}"
            deg_since_after_pair = (year_of_interest, f"{year_of_interest}_no_degradation_since_{specific_year_plus1}")
            dist_since_after = f"{year_of_interest}_disturbance_since_{specific_year_plus1}"
            dist_since_after_pair = (year_of_interest, f"{year_of_interest}_no_disturbance_since_{specific_year_plus1}")

            other_requirements.append((deg_since_specific, deg_since_specific_pair))
            other_requirements.append((dist_since_specific, dist_since_specific_pair))
            other_requirements.append((deg_since_after, deg_since_after_pair))
            other_requirements.append((dist_since_after, dist_since_after_pair))

            defor_since_specific = f"{year_of_interest}_deforestation_since_{specific_year}"
            defor_since_after = f"{year_of_interest}_deforestation_since_{specific_year_plus1}"
            calculation_note = f"Calculated as {defor_since_specific} - {defor_since_after}"

        elif disturbance_type == "3":  # Total disturbance effect
            selected_difference = f"{year_of_interest}_effect_of_disturbance_in_{specific_year}"

            # Define required difference components
            since_specific = f"{year_of_interest}_disturbance_since_{specific_year}"
            since_specific_pair = (year_of_interest, f"{year_of_interest}_no_disturbance_since_{specific_year}")

            since_after = f"{year_of_interest}_disturbance_since_{specific_year_plus1}"
            since_after_pair = (year_of_interest, f"{year_of_interest}_no_disturbance_since_{specific_year_plus1}")

            other_requirements.append((since_specific, since_specific_pair))
            other_requirements.append((since_after, since_after_pair))
            calculation_note = f"Calculated as {since_specific} - {since_after}"

    elif baseline_type == "5":  # Effect in the same year
        if disturbance_type == "1":  # Degradation
            selected_difference = f"{year_of_interest}_effect_of_degradation_in_{year_of_interest}"

            # This is a copy operation
            source_difference = f"{year_of_interest}_degradation_since_{year_of_interest}"
            source_pair = (year_of_interest, f"{year_of_interest}_no_degradation_since_{year_of_interest}")

            other_requirements.append((source_difference, source_pair))
            calculation_note = f"Copy and rename {source_difference}"

        elif disturbance_type == "2":  # Deforestation
            selected_difference = f"{year_of_interest}_effect_of_deforestation_in_{year_of_interest}"

            # Define components for same-year deforestation
            deg_same_year = f"{year_of_interest}_degradation_since_{year_of_interest}"
            deg_same_year_pair = (year_of_interest, f"{year_of_interest}_no_degradation_since_{year_of_interest}")
            dist_same_year = f"{year_of_interest}_disturbance_since_{year_of_interest}"
            dist_same_year_pair = (year_of_interest, f"{year_of_interest}_no_disturbance_since_{year_of_interest}")

            other_requirements.append((deg_same_year, deg_same_year_pair))
            other_requirements.append((dist_same_year, dist_same_year_pair))

            defor_same_year = f"{year_of_interest}_deforestation_since_{year_of_interest}"
            calculation_note = f"Copy and rename {defor_same_year}"

        elif disturbance_type == "3":  # Total disturbance in same year
            selected_difference = f"{year_of_interest}_effect_of_disturbance_in_{year_of_interest}"

            # This is a copy operation
            source_difference = f"{year_of_interest}_disturbance_since_{year_of_interest}"
            source_pair = (year_of_interest, f"{year_of_interest}_no_disturbance_since_{year_of_interest}")

            other_requirements.append((source_difference, source_pair))
            calculation_note = f"Copy and rename {source_difference}"

    else:
        print("\nError: Invalid baseline type selection.")
        return None

    # Build result display
    result_text = []
    result_text.append("\n" + "="*50)
    result_text.append(f"\nSELECTED DIFFERENCE: {selected_difference}")

    # Add required differences and their scenario pairs
    if other_requirements:
        result_text.append(f"\nRequired difference and scenario pairs:")
        for diff, pair in other_requirements:
            result_text.append(f"'{diff}' {pair}")

        if calculation_note:
            result_text.append(f"\n{calculation_note}")
    # Only include scenario pair when no other requirements
    elif scenario_pair:
        result_text.append(f"\nScenario pair required: {scenario_pair}")

    result_text.append("\n" + "="*50)

    # Print results with fewer new lines
    print("\n\n")
    print("\n".join(result_text))

    # Return appropriate values
    if is_specific_effects_calculation:
        return selected_difference, other_requirements
    else:
        return selected_difference, scenario_pair, other_requirements

# Run the function
if use_tool:
  if __name__ == "__main__":
      select_forest_scenarios()

## No degradation scenarios

In [ ]:
# These alternate scenarios remove degradation for specific time ranges

# Define ranges for 'no degradation' scenarios
define_no_degradation_scenarios = True

# No degradation ranges as tuples of (start_year, end_year)
no_degradation_ranges = [
    (1993, 2021),
    (1996, 2024),
    # (1997, 2024),
    # (1998, 2024),
    # (1999, 2024),
    # (2000, 2024),
    # (2001, 2024),
    # (2002, 2024),
    # (2003, 2024),
    # (2004, 2024),
    # (2005, 2024),
    # (2006, 2024),
    # (2007, 2024),
    # (2008, 2024),
    # (2009, 2024),
    # (2010, 2024),
    # (2011, 2024),
    # (2012, 2024),
    # (2013, 2024),
    # (2014, 2024),
    # (2015, 2024),
    # (2016, 2024),
    # (2017, 2024),
    # (2018, 2024),
    # (2019, 2024),
    # (2020, 2024),
    # (2021, 2024),
    # (2022, 2024),
    # (2023, 2024),
    # (2024, 2024),
]

# Create a feature list for 'no degradation' scenarios
if define_no_degradation_scenarios:
  for start_year, end_year in no_degradation_ranges:
    assert end_year <= last_feature_year, "End years must be at or before the last feature year."
    assert end_year >= minimum_yearly_scenario, "End years must be at or after the minimum yearly scenario."
    assert start_year >= first_feature_year, "Start years must be at or after the first feature year."
    assert start_year >= end_year - model_scenario_year_range, "Start years must be within the model scenario range of the end year."
    assert start_year <= end_year, "The start year must less than or equal to the end year."

    # Determine base features based on the end year of the range
    scenario_features_csv = join(scenarios_model_dir, f"{end_year}.csv")
    base_features = pd.Series.tolist(pd.read_csv(scenario_features_csv).iloc[:,0])

    no_degradation_features = []
    for scenario_feature in base_features:
      if "disturbance_with_edge_effects" in scenario_feature:
        scenario_feature_year = int(scenario_feature[-4:])
        # Replace disturbance feature if it falls within the specified range
        if scenario_feature_year >= start_year:
          no_degradation_features.append(minimum_disturbance_name)
        else:
          no_degradation_features.append(scenario_feature)
      else:
        no_degradation_features.append(scenario_feature)

    no_degradation_scenario_filename = f"{end_year}_no_degradation_since_{start_year}.csv"
    no_degradation_scenario_path = join(scenarios_model_dir, no_degradation_scenario_filename)
    pd.DataFrame(no_degradation_features).to_csv(no_degradation_scenario_path, index=False)
    print(f"Feature list for a scenario without degradation between {start_year} and {end_year} exported to {no_degradation_scenario_filename}.")
else:
  print("The 'no degradation' scenarios are not enabled.")

## No disturbance scenarios

In [ ]:
# These alternate scenarios remove both disturbance and deforestation for specific time ranges

# Define ranges for 'no disturbance' scenarios
define_no_disturbance_scenarios = True

# No disturbance ranges as tuples of (start_year, end_year)
no_disturbance_ranges = [
    (1993, 2021),
    (1996, 2024),
    (1997, 2024),
    (1998, 2024),
    (1999, 2024),
    (2000, 2024),
    (2001, 2024),
    (2002, 2024),
    (2003, 2024),
    (2004, 2024),
    (2005, 2024),
    (2006, 2024),
    (2007, 2024),
    (2008, 2024),
    (2009, 2024),
    (2010, 2024),
    (2011, 2024),
    (2012, 2024),
    (2013, 2024),
    (2014, 2024),
    (2015, 2024),
    (2016, 2024),
    (2017, 2024),
    (2018, 2024),
    (2019, 2024),
    (2020, 2024),
    (2021, 2024),
    (2022, 2024),
    (2023, 2024),
    (2024, 2024),
]

# Create a feature list for 'no disturbance' scenarios
if define_no_disturbance_scenarios:
  for start_year, end_year in no_disturbance_ranges:
    assert end_year <= last_feature_year, "End years must be at or before the last feature year."
    assert end_year >= minimum_yearly_scenario, "End years must be at or after the minimum yearly scenario."
    assert start_year >= first_feature_year, "Start years must be at or after the first feature year."
    assert start_year >= end_year - model_scenario_year_range, "Start years must be within the model scenario range of the end year."
    assert start_year <= end_year, "The start year must less than or equal to the end year."

    # Determine base features based on the end year of the range
    scenario_features_csv = join(scenarios_model_dir, f"{end_year}.csv")
    base_features = pd.Series.tolist(pd.read_csv(scenario_features_csv).iloc[:,0])

    no_disturbance_features = []
    for scenario_feature in base_features:
      if "disturbance_with_edge_effects" in scenario_feature:
        scenario_feature_year = int(scenario_feature[-4:])
        # Replace disturbance feature if it falls within the specified range
        if scenario_feature_year >= start_year: no_disturbance_features.append(minimum_disturbance_name)
        else: no_disturbance_features.append(scenario_feature)
      elif "forest_with_edge_effects" in scenario_feature:
        scenario_feature_year = int(scenario_feature[-4:])
        forest_edge_effects_year = start_year - 1
        alternate_forest_edge_effects = scenario_feature[:-4] + str(forest_edge_effects_year)
        # Replace forest feature if it falls within the specified range
        if scenario_feature_year > forest_edge_effects_year: no_disturbance_features.append(alternate_forest_edge_effects)
        else: no_disturbance_features.append(scenario_feature)
      else: no_disturbance_features.append(scenario_feature)

    no_disturbance_scenario_filename = f"{end_year}_no_disturbance_since_{start_year}.csv"
    no_disturbance_scenario_path = join(scenarios_model_dir, no_disturbance_scenario_filename)
    pd.DataFrame(no_disturbance_features).to_csv(no_disturbance_scenario_path, index=False)
    print(f"Feature list for a scenario without disturbance between {start_year} and {end_year} exported to {no_disturbance_scenario_filename}.")
else:
  print("The 'no disturbance' scenarios are not enabled.")

## Oldgrowth scenarios

In [ ]:
# List of land-use features. All but one are 'redundant',
# One should be selected as 'oldgrowth_feature'.
print("oldgrowth_redundant_features = [")
for feature in model_features:
  if "lu_" in feature:
    print(f'  "{feature}",')
print("]")

In [ ]:
# These alternate scenarios simulate old-growth forest using a proxy area specified by the user.
# Forest extent (i.e. 'no deforestation') can be set from yearly feature, or all historic / potential forest area.
# A second version of the scenario without the oldgrowth proxy will be created for comparison.
# In rare cases, areas with unexpectedly high AGBD will have a lower AGBD estimate with the oldgrowth proxy,
# and the highest estimate will be used for that pixel.

define_oldgrowth_scenarios = True
oldgrowth_yearly_scenarios = [
    2021,
    last_feature_year
]

for year in oldgrowth_yearly_scenarios:
  assert year in final_feature_years, "Years in 'oldgrowth_yearly_scenarios' must be available in the final yearly features."

# oldgrowth_all_land will be created for each year in oldgrowth_yearly_scenarios
simulate_oldgrowth_all_land = True

# The feature that best indicates oldgrowth to the model, e.g. certain protected areas
# This will be modified to cover the entire template area for the old-growth scenarios
oldgrowth_feature = 'lu_old-growth_protected_areas_with_edge_effects'

# Some features may confound the old-growth proxy, e.g. protected areas that are not known to be old-growth
# These will be removed for the old-growth scenarios
oldgrowth_redundant_features = [
  "lu_ais_with_edge_effects",
  "lu_berkelah_jerantut_with_edge_effects",
  "lu_berkelah_kuantan_with_edge_effects",
  "lu_berkelah_temerloh_with_edge_effects",
  "lu_old-growth_protected_areas_with_edge_effects",
  "lu_remen_chereh_with_edge_effects",
  "lu_tekai_tembeling_with_edge_effects",
  "lu_tekam_with_edge_effects",
  "lu_yong_lipis_with_edge_effects",
  "lu_yong_with_edge_effects",
]

if define_oldgrowth_scenarios:
  # Expand the oldgrowth feature to the entire template area
  oldgrowth_feature_all_dir = join(features_dir, f"{oldgrowth_feature}_all.tif")
  if not exists(oldgrowth_feature_all_dir):
    oldgrowth_feature_dir = join(features_dir, f"{oldgrowth_feature}.tif")
    oldgrowth_feature_array = gdal.Open(oldgrowth_feature_dir).ReadAsArray()
    oldgrowth_feature_max_value = oldgrowth_feature_array.max()
    print(f"The maximum value for the oldgrowth feature '{oldgrowth_feature}' is {oldgrowth_feature_max_value}.")
    oldgrowth_feature_all_array = np.where(oldgrowth_feature_array, oldgrowth_feature_max_value, oldgrowth_feature_max_value)
    oldgrowth_feature_all_dir = join(features_dir, f"{oldgrowth_feature}_all.tif")
    export_array_as_tif(oldgrowth_feature_all_array, oldgrowth_feature_all_dir, template = oldgrowth_feature_dir)
    print(f"The oldgrowth proxy {oldgrowth_feature} has been expanded to the entire scenario area")
    print(f"And exported to {oldgrowth_feature_all_dir}")
  else: print(f"The oldgrowth feature '{oldgrowth_feature}_all.tif' already exists.\n")

  # Remove the redundant features from the oldgrowth template area
  for redundant_feature in oldgrowth_redundant_features:
    redundant_feature_none_dir = join(features_dir, f"{redundant_feature}_none.tif")
    if not exists(redundant_feature_none_dir):
      redundant_feature_dir = join(features_dir, f"{redundant_feature}.tif")
      redundant_feature_array = gdal.Open(redundant_feature_dir).ReadAsArray()
      redundant_feature_min_value = redundant_feature_array.min()
      print(f"The minimum value for the redundant feature {redundant_feature} is {redundant_feature_min_value}.")
      redundant_feature_none_array = np.where(redundant_feature_array, redundant_feature_min_value, redundant_feature_min_value)
      export_array_as_tif(redundant_feature_none_array, redundant_feature_none_dir, redundant_feature_dir)
      print(f"The oldgrowth redundant feature {oldgrowth_feature} has been removed from the entire template area")
      print(f"And exported to {oldgrowth_feature_all_dir}.")
    else: print(f"The oldgrowth redundant feature '{redundant_feature}_none.tif' already exists.\n")

  # Generate 'oldgrowth' scenarios for each specified year (i.e. forest extent in that year)
  for year in oldgrowth_yearly_scenarios:
    # Two oldgrowth feature sets created
    oldgrowth_features_1 = []
    oldgrowth_features_2 = []
    old_growth_scenario_year_diff = last_feature_year - year
    for scenario_feature in most_recent_scenario_features:
      if "disturbance_with_edge_effects" in scenario_feature:
        feature_1 = feature_2 = minimum_disturbance_name
      elif "forest_with_edge_effects" in scenario_feature:
        scenario_feature_year = int(scenario_feature[-4:])
        new_year = (scenario_feature_year-old_growth_scenario_year_diff > first_feature_year) and \
                  f"forest_with_edge_effects_{scenario_feature_year-old_growth_scenario_year_diff}" or \
                  f"forest_with_edge_effects_{first_feature_year}"
        feature_1 = feature_2 = new_year
      # Only change oldgrowth feature and redundant feature in one version
      elif scenario_feature == oldgrowth_feature:
        feature_1 = f"{scenario_feature}_all"
        feature_2 = scenario_feature
      elif scenario_feature in oldgrowth_redundant_features:
        feature_1 = f"{scenario_feature}_none"
        feature_2 = scenario_feature
      else: feature_1 = feature_2 = scenario_feature
      oldgrowth_features_1.append(feature_1)
      oldgrowth_features_2.append(feature_2)

    # Compare feature lists and save appropriate CSVs
    if oldgrowth_features_1 == oldgrowth_features_2:
      # Lists are identical, only save version 1
      filename = f"{year}_oldgrowth_1.csv"
      pd.DataFrame(oldgrowth_features_1).to_csv(join(scenarios_model_dir, filename), index=False)
      print(f"Feature lists were identical, only saved {filename}")
    else:
      # Lists are different, save both versions
      for suffix, features in [("1", oldgrowth_features_1), ("2", oldgrowth_features_2)]:
        filename = f"{year}_oldgrowth_{suffix}.csv"
        pd.DataFrame(features).to_csv(join(scenarios_model_dir, filename), index=False)
      print(f"Feature lists for scenarios where all forest in {year} was old-growth")
      print(f"have been exported to {year}_oldgrowth_1.csv and {year}_oldgrowth_2.csv.\n")

  # Generate 'oldgrowth_all_land' features and scenarios
  if simulate_oldgrowth_all_land:
    for year in oldgrowth_yearly_scenarios:
      oldgrowth_all_land_name = f"{year}_oldgrowth_all_land"
      # Create a forest feature for all land that exists in the first feature year (e.g. 1990)
      forest_oldgrowth_all_land_name = f"forest_with_edge_effects_{oldgrowth_all_land_name}"
      forest_oldgrowth_all_land_path = join(features_dir, f"{forest_oldgrowth_all_land_name}.tif")
      annual_changes_filename = f"tmf_AnnualChanges_Dec{year}.tif"
      annual_changes_path = join(feature_resampled_dir, annual_changes_filename)
      if not exists(forest_oldgrowth_all_land_path):
        if exists(annual_changes_path):
          first_annual_changes_array = gdal.Open(annual_changes_path).ReadAsArray()
          # Convert all water values to '0' and non-water values to '1'
          forest_oldgrowth_all_land_array = np.where(first_annual_changes_array == 5, 0, 1)
          # Set smoothing kernel and precision
          export_array_as_tif(edge_effects(forest_oldgrowth_all_land_array), forest_oldgrowth_all_land_path)
          print(f"{forest_oldgrowth_all_land_name} has been created and saved to\n{features_dir}\n")
        else: print(f"The TMF annual changes {year} raster needed for '{year}_oldgrowth_all_land' is not in the indicated directory:\n{annual_changes_path}\n")
      print(f"{forest_oldgrowth_all_land_name} already exists in\n{features_dir}\n")

      if exists(forest_oldgrowth_all_land_path):
        # Two oldgrowth feature sets created, both removing disturbance from all years
        # oldgrowth_all_land_1 also adds the land-use oldgrowth proxy
        # During masking, the version with highest AGBD value is selected.
        oldgrowth_all_features_1 = []
        oldgrowth_all_features_2 = []
        for scenario_feature in most_recent_scenario_features:
          if "disturbance_with_edge_effects" in scenario_feature:
            feature_1 = feature_2 = minimum_disturbance_name
          elif "forest_with_edge_effects" in scenario_feature:
            feature_1 = feature_2 = forest_oldgrowth_all_land_name
          # Only change oldgrowth feature and redundant feature in one version
          elif scenario_feature == oldgrowth_feature:
            feature_1 = f"{scenario_feature}_all"
            feature_2 = scenario_feature
          elif scenario_feature in oldgrowth_redundant_features:
            feature_1 = f"{scenario_feature}_none"
            feature_2 = scenario_feature
          else: feature_1 = feature_2 = scenario_feature
          oldgrowth_all_features_1.append(feature_1)
          oldgrowth_all_features_2.append(feature_2)

        # Compare feature lists and save appropriate CSVs
        if oldgrowth_all_features_1 == oldgrowth_all_features_2:
          # Lists are identical, only save version 1
          filename = f"{oldgrowth_all_land_name}_1.csv"
          pd.DataFrame(oldgrowth_all_features_1).to_csv(join(scenarios_model_dir, filename), index=False)
          print(f"Feature lists were identical, only saved {filename}")
        else:
          # Lists are different, save both versions
          for suffix, features in [("1", oldgrowth_all_features_1), ("2", oldgrowth_all_features_2)]:
            filename = f"{oldgrowth_all_land_name}_{suffix}.csv"
            pd.DataFrame(features).to_csv(join(scenarios_model_dir, filename), index=False)
          print(f"Feature lists for {oldgrowth_all_land_name} have been exported to {oldgrowth_all_land_name}_1.csv and {oldgrowth_all_land_name}_2.csv.\n")

      # Create an all-land forest mask for 'all land' oldgrowth
      oldgrowth_all_mask_path = join(masks_dir, f"mask_forest_{oldgrowth_all_land_name}.tif")
      if not exists(oldgrowth_all_mask_path):
        first_annual_changes_array = gdal.Open(annual_changes_path).ReadAsArray()
        # Convert all water values to 'nodata' and non-water values to '1'
        oldgrowth_all_mask_array = np.where(first_annual_changes_array == 5, nodatavalue, 1)
        export_array_as_tif(oldgrowth_all_mask_array, oldgrowth_all_mask_path)
        print(f"A mask for {oldgrowth_all_land_name} has been created at\n{oldgrowth_all_mask_path}\n")
      else: print(f"A mask for {oldgrowth_all_land_name} already exists at\n{oldgrowth_all_mask_path}\n")

else: print("Old-growth scenarios are not enabled.")

## Area-based disturbance

In [ ]:
# Use polygons to select areas for alternate scenarios of area-based disturbance
define_area_based_disturbance = True

if define_area_based_disturbance:
  # Exclude existing polygons from search
  polygons_to_exclude = ['project_area.gpkg', 'project_area_buffered_bbox.gpkg', 'gedi_area.gpkg', 'template.gpkg']
  print("# Modify this dictionary by:")
  print("# 1) Commenting out any polygons not being used for disturbance.")
  print("# 2) Changing the type from 'deforestation' to 'degradation' if necessary.")
  print("# 3) Changing the tuple years from 'range' to 'discrete' to specify individual years.")
  print("# 4) If years are discrete, add one or more. If a range, add the start and end year.")
  print("# 5) Changing the alternate scenario year for each area if needed.")
  print("# 6) Copy and paste lines for multiple scenarios with the same area (different disturbance types, different years).")
  print("# Remember to ensure all keys are unique - if you copy and paste, manually change the keys.\n")

  # Exclude existing polygons from search
  polygons_to_exclude = ['project_area.gpkg', 'gedi_area.gpkg', 'template.gpkg']
  exclude_lu_polygons = False

  print("disturbance_polygons = {")
  index = 1
  first_disturbance_year = last_feature_year - model_scenario_year_range
  for polygon in sorted(os.listdir(polygons_dir)):
    if polygon not in polygons_to_exclude and 'inverse' not in polygon and 'buffered' not in polygon:
      if not exclude_lu_polygons:
        print(f"    {index}: ['{polygon[:-5]}', 'deforestation','range', ({first_disturbance_year}, {last_feature_year}), {last_feature_year}],")
        index += 1
      if exclude_lu_polygons and 'lu_' not in polygon:
        print(f"    {index}: ['{polygon[:-5]}', 'deforestation','range', ({first_disturbance_year}, {last_feature_year}), {last_feature_year}],")
        index += 1
  print("}\n")

else: print("Area-based disturbance scenarios are not enabled.")

In [ ]:
# The alternate year is set to 2024.
# The years for alternate area-based disturbance can be between 1996 and 2024

# Modify this dictionary by:
# 1) Commenting out any polygons not being used for disturbance.
# 2) Changing the type from 'deforestation' to 'degradation' if necessary.
# 3) Changing the tuple years from 'range' to 'discrete' to specify individual years.
# 4) If years are discrete, add one or more. If a range, add the start and end year.
# 5) Copy and paste lines for multiple scenarios with the same area (different disturbance types, different years).
# Remember to ensure all keys are unique - if you copy and paste, manually change the keys.

if define_area_based_disturbance:
  disturbance_polygons = {
      3: ['road_mat_daling', 'deforestation','range', (2023, 2024), 2024],
  }

  # Validate disturbance types, year types and available years.
  for area_index, value in disturbance_polygons.items():
      polygon_name = value[0]
      disturbance_type = value[1]
      year_type, years_data = value[2], value[3]
      alternate_scenario_year = value[4]

      # Calculate first available disturbance year for this area's alternate scenario year
      first_disturbance_year = alternate_scenario_year - model_scenario_year_range

      # Validate alternate scenario year
      assert alternate_scenario_year >= minimum_yearly_scenario, f"Alternate scenario year for {polygon_name} must be between {minimum_yearly_scenario} and {last_feature_year}."
      assert alternate_scenario_year <= last_feature_year, f"Alternate scenario year for {polygon_name} must be between {minimum_yearly_scenario} and {last_feature_year}."
      # Validate disturbance types and year types
      assert disturbance_type in ['deforestation', 'degradation'], f"Disturbance type for {polygon_name} must be 'deforestation' or 'degradation'."
      if year_type == 'range':
        start_year, end_year = years_data
        assert start_year <= end_year, f"The start year for {polygon_name} {disturbance_type} must be before the end year."

      # Validate deforestation constraints
      if disturbance_type == 'deforestation':
        assert year_type == 'range', f"Year type for {polygon_name} deforestation must be 'range'."
        assert end_year == alternate_scenario_year, f"Deforestation in {polygon_name} must end in the alternate scenario year {alternate_scenario_year}. Deforestation is considered permanent land-cover change."
        assert start_year >= first_disturbance_year, f"The start year for deforestation in {polygon_name} must be >= the first available disturbance year {first_disturbance_year}."
        all_years = list(range(start_year, end_year + 1))

      # Validate degradation constraints
      if disturbance_type == 'degradation':
        assert year_type in ['range', 'discrete'], f"Year type for {polygon_name} degradation must be 'range' or 'discrete'."
        if year_type == 'range':
            all_years = list(range(start_year, end_year + 1))
        else: all_years = list(years_data)
        for year in all_years:
            assert year <= alternate_scenario_year, f"Years for {polygon_name} degradation (check {year}) must be <= the alternate scenario year {alternate_scenario_year}."
            assert year >= first_disturbance_year, f"Years for {polygon_name} degradation (check {year}) must >= the first available disturbance year {first_disturbance_year}."

      # Simplify dictionary
      disturbance_polygons[area_index] = [polygon_name, disturbance_type, all_years, alternate_scenario_year]

  print("The 'disturbance_polygons' dictionary is valid.")
else: print("Area-based disturbance scenarios are not enabled.")

In [ ]:
if define_area_based_disturbance:
  # Define the projects CRS to check the area polygon matches
  crs_epsg = 4326
  # This setting buffers any deforestation area to add degradation around it.
  # At least 1 pixel distance (e.g. 30 m) might be realistic. Otherwise set to None.
  buffer_distance_metres = 30

  # Define a temporary directory for copying binary rasters and burning the area's polygon
  binary_temp_dir = join(scenarios_model_dir, 'binary_temp')
  makedirs(binary_temp_dir, exist_ok=True)

  # Calculate progress totals before processing - simplified counting
  total_areas = len(disturbance_polygons)
  total_rasters = 0

  for area_index, parameters in disturbance_polygons.items():
      polygon_name = parameters[0]
      disturbance_type = parameters[1]
      disturbance_years = parameters[2]

      # Simplified raster counting: degradation = years x 1, deforestation = years x 3
      if disturbance_type == 'degradation':
          area_raster_count = len(disturbance_years) * 1
      elif disturbance_type == 'deforestation':
          area_raster_count = len(disturbance_years) * 3  # disturbance + forest + mask

      total_rasters += area_raster_count

  # Progress indicators
  area_progress_index, area_progress_label = 0, widgets.Label(value=f"Area progress: 0 / {total_areas}")
  display(area_progress_label)
  raster_progress_index, raster_progress_label = 0, widgets.Label(value=f"Raster progress: 0 / {total_rasters}")
  display(raster_progress_label)

  for area_index, parameters in disturbance_polygons.items():

      # Extract alternate area-based disturbance parameters
      area_disturbance_features = []
      polygon_name = parameters[0]
      disturbance_type = parameters[1]
      disturbance_years = parameters[2]
      alternate_scenario_year = parameters[3]

      # Determine base features by the alternate scenario's year for this area
      alternate_year_scenario_csv = join(scenarios_model_dir, f"{alternate_scenario_year}.csv")
      base_features = pd.Series.tolist(pd.read_csv(alternate_year_scenario_csv).iloc[:,0])

      # Define area polygon
      area_polygon_path = join(polygons_dir, f"{polygon_name}.gpkg")
      if disturbance_type == 'deforestation' and buffer_distance_metres:
        area_buffered_path = join(polygons_dir, f"{polygon_name}_buffered_{buffer_distance_metres}.gpkg")
        if not exists(area_buffered_path):
          area_polygon = gpd.read_file(join(polygons_dir, f"{polygon_name}.gpkg"))
          if area_polygon.crs.to_epsg() == crs_epsg:
            # Suppress warning about not being a geographic CRS, as we account for this.
            # However larger buffers or project areas near the poles might still need to be converted.
            warnings.filterwarnings("ignore", category=UserWarning)
            # Get the centroid of the project polygon
            area_polygon_centroid = area_polygon.centroid.values[0]
            # Convert the buffer distance from meters to decimal degrees based on the location at the centroid
            buffer_distance_degrees = buffer_distance_metres / (111320 * abs(math.cos(math.radians(area_polygon_centroid.y))))
            # Buffer the polygon and save
            area_polygon_buffered = area_polygon.buffer(buffer_distance_degrees)
            gdf = gpd.GeoDataFrame(geometry=area_polygon_buffered, crs=f"EPSG:{crs_epsg}")
            gdf.to_file(area_buffered_path, driver='GPKG')
            print(f"Buffered the project area to {buffer_distance_metres} and exported to the polygons directory.")
          else: print(f"Reproject {polygon_name}.gpkg to EPSG:4326.")
      else: area_buffered_path = None

      for scenario_feature in base_features:
          # Handle disturbance features if years match
          if "disturbance" in scenario_feature:
              scenario_feature_year = int(scenario_feature[-4:])
              if scenario_feature_year in disturbance_years:
                # Define the new feature name and path
                if disturbance_type == 'deforestation': area_disturbance_name = f"{scenario_feature}_{polygon_name}_deforestation_{buffer_distance_metres}m_buffer"
                else: area_disturbance_name = f"{scenario_feature}_{polygon_name}_degradation"
                area_based_disturbance_path = join(features_dir,  f"{area_disturbance_name}.tif")

                if not exists(area_based_disturbance_path):
                  # Copy the disturbance binary raster for burning '1' to the the polygon area
                  binary_raster_name = f"disturbance_binary_{scenario_feature_year}.tif"
                  binary_raster_path = join(feature_binary_dir, binary_raster_name)
                  binary_raster_temp_path = join(binary_temp_dir, binary_raster_name)
                  copyfile(binary_raster_path, binary_raster_temp_path)
                  if area_buffered_path: burn_polygon_to_raster(binary_raster_temp_path, area_buffered_path, fixed_value=1, all_touched=True)
                  else: burn_polygon_to_raster(binary_raster_temp_path, area_polygon_path, fixed_value=1, all_touched=True)
                  # Open and apply edge effects to the burned array, then export the raster
                  binary_burned_array = gdal.Open(binary_raster_temp_path).ReadAsArray()
                  export_array_as_tif(edge_effects(binary_burned_array), area_based_disturbance_path)

                area_disturbance_features.append(area_disturbance_name)
                raster_progress_index += 1
                raster_progress_label.value = f"Raster progress: {raster_progress_index} / {total_rasters}"
              else: area_disturbance_features.append(scenario_feature)

          # Handle forest features if disturbance type is deforestation and years match
          elif "forest_with_edge_effects" in scenario_feature:
              if disturbance_type == 'deforestation':
                  scenario_feature_year = int(scenario_feature[-4:])
                  if scenario_feature_year in disturbance_years:

                    # Define the new feature name and path
                    area_disturbance_name = f"{scenario_feature}_{polygon_name}_deforestation"
                    area_based_disturbance_path = join(features_dir, f"{area_disturbance_name}.tif")

                    if not exists(area_based_disturbance_path):
                      # Copy the forest binary raster for burning '0' to the the polygon area
                      binary_raster_name = f"forest_binary_{scenario_feature_year}.tif"
                      binary_raster_path = join(feature_binary_dir, binary_raster_name)
                      binary_raster_temp_path = join(binary_temp_dir, binary_raster_name)
                      copyfile(binary_raster_path, binary_raster_temp_path)
                      burn_polygon_to_raster(binary_raster_temp_path, area_polygon_path, fixed_value=0, all_touched=True)
                      # Open and apply edge effects to the burned array, then export the raster
                      binary_burned_array = gdal.Open(binary_raster_temp_path).ReadAsArray()
                      export_array_as_tif(edge_effects(binary_burned_array), area_based_disturbance_path)

                    area_disturbance_features.append(area_disturbance_name)
                    raster_progress_index += 1
                    raster_progress_label.value = f"Raster progress: {raster_progress_index} / {total_rasters}"
                  else: area_disturbance_features.append(scenario_feature)
              else: area_disturbance_features.append(scenario_feature)
          else: area_disturbance_features.append(scenario_feature)

      # Add name affix based on whether years are a range or discrete
      if disturbance_type == "deforestation": year_affix = f'_{min(disturbance_years)}'
      else:
        if len(disturbance_years) != (max(disturbance_years) - min(disturbance_years) + 1):
            sorted_years = sorted(disturbance_years)
            parts, start = [], sorted_years[0]
            for i, year in enumerate(sorted_years[1:] + [None], 1):
                if year != sorted_years[i-1] + 1:
                    end = sorted_years[i-1]
                    parts.append(f"{start}-{end}" if start != end else str(start))
                    start = year
            year_affix = "_" + "_".join(parts)
        else: year_affix = f'_{min(disturbance_years)}-{max(disturbance_years)}'

      if disturbance_type == 'deforestation':
        area_disturbance_scenario_name = f"{alternate_scenario_year}_{polygon_name}_deforestation{year_affix}_{buffer_distance_metres}m_degradation_buffer"
        # Create a new forest mask for the area-based disturbance scenario
        mask_raster_path = join(masks_dir, f"mask_forest_{alternate_scenario_year}_{polygon_name}_deforestation.tif")
        if not exists(mask_raster_path):
          # Ensure original forest binary is copied to temp and burned with polygon
          scenario_year_forest_binary_path = join(binary_temp_dir, f"forest_binary_{alternate_scenario_year}.tif")
          forest_binary_source = join(feature_binary_dir, f"forest_binary_{alternate_scenario_year}.tif")
          copyfile(forest_binary_source, scenario_year_forest_binary_path)
          burn_polygon_to_raster(scenario_year_forest_binary_path, area_polygon_path, fixed_value=0, all_touched=True)
          # Create mask from burned forest data
          scenario_year_forest_binary_array = gdal.Open(scenario_year_forest_binary_path).ReadAsArray()
          mask_array = np.where(scenario_year_forest_binary_array == 0, nodatavalue, 1)
          export_array_as_tif(mask_array, mask_raster_path)
          print(f"A mask raster has been created for {area_disturbance_scenario_name}.")
        raster_progress_index += 1
        raster_progress_label.value = f"Raster progress: {raster_progress_index} / {total_rasters}"

      else: area_disturbance_scenario_name = f"{alternate_scenario_year}_{polygon_name}_degradation{year_affix}"

      # Clear temporary binary raster folder
      for temp_file in os.listdir(binary_temp_dir): os.remove(join(binary_temp_dir, temp_file))

      # Export the alternate area-based disturbance scenario
      no_degradation_scenario_path = join(scenarios_model_dir, f"{area_disturbance_scenario_name}.csv")
      pd.DataFrame(area_disturbance_features).to_csv(no_degradation_scenario_path, index=False)
      print(f"Feature list for {area_disturbance_scenario_name} has been exported.")

      # Update area progress
      area_progress_index += 1
      area_progress_label.value = f"Area progress: {area_progress_index} / {total_areas}"

  print("\nAlternate area-based disturbance scenarios complete.")
else: print("Area-based disturbance scenarios are not enabled.")

# Feature verification (optional)

In [ ]:
# Check all features in scenario .csvs exist
scenario_csv_list = []
all_features_exist = True # Changes to false if feature missing
for csv in os.listdir(scenarios_model_dir):
  if csv.endswith('.csv'):
    csv_dir = join(scenarios_model_dir, csv)
    csv_feature_list = pd.Series.tolist(pd.read_csv(csv_dir).iloc[:,0])
    csv_feature_dir_list = []
    for csv_feature in csv_feature_list:
      if csv_feature not in covariates: csv_feature_dir_list.append(f"{features_dir}/{csv_feature}.tif")
    for feature in csv_feature_dir_list:
      if not exists(feature):
        all_features_exist = False
        print(f"The following feature is missing:\n{feature}\n and is required for the scenario '{csv[:-4]}'")

if all_features_exist: print("All required features are present.")
print("Covariate features e.g. 'beam' and 'sensitivity' will be added at the prediction stage.")

# Check all features against template dimensions
scenario_template = gdal.Open(template_tif_path)
scenario_template_dimensions, scenario_template_projection = scenario_template.GetGeoTransform(), scenario_template.GetProjection()
feature_issue = False
for feature in os.listdir(features_dir):
  if feature.endswith('.tif'):
    feature_dir = join(features_dir, feature)
    feature_open = gdal.Open(feature_dir)
    feature_dimensions, feature_projection = feature_open.GetGeoTransform(), feature_open.GetProjection()
    if feature_dimensions != scenario_template_dimensions:
      print(f"{feature} dimensions:\n{feature_dimensions}\ndo not match the scenario template dimensions:\n{scenario_template_dimensions}\n")
      feature_issue = True
    if feature_projection != scenario_template_projection:
      print(f"{feature} projection:\n{feature_projection}\ndoes not match the scenario template projection:\n{scenario_template_projection}\n\n")
      feature_issue = True

if not feature_issue: print(f"All features in the following directory have the correct dimensions and projection:\n{features_dir}")
else: print("Correct and / or resample the feature(s).")

# Tiling

In [ ]:
# Load the model scenario features for template tile creation
model_scenario_features = pd.Series.tolist(pd.read_csv(model_scenario_dir).iloc[:,0])
model_scenario_features_dirs = [features_dir + '/' + feature + '.tif' for feature in model_features]
# Create a template feature array from the first feature that isn't a covariate (these are created later)
template_base_path = next(r for r in model_scenario_features_dirs if all(c not in r for c in covariates))
template_base = gdal.Open(template_base_path)
template_base_array = template_base.ReadAsArray()
template_base_xsize, template_base_ysize = template_base.GetRasterBand(1).XSize, template_base.GetRasterBand(1).YSize
print(f"The template feature is {template_base_xsize} x {template_base_ysize} pixels.")

# Check existing tile parameters
template_tile_list = []
for file in os.listdir(tile_templates_dir):
  if file.endswith('.tif') and file[:13] == 'template_tile':
    template_tile_list.append(file)
n_tiles_exist = len(template_tile_list)

if n_tiles_exist < 1: print("There are currently no template tiles. Run the next section.")
if n_tiles_exist >= 1:
  tile_size_y_rounded_exist = gdal.Open(join(tile_templates_dir,'template_tile_1.tif'))
  tile_size_y_rounded_exist_y = tile_size_y_rounded_exist.GetRasterBand(1).YSize
  tile_size_y_remainder_exist = gdal.Open(join(tile_templates_dir,f'template_tile_{n_tiles_exist}.tif'))
  tile_size_y_remainder_exist_y = tile_size_y_remainder_exist.GetRasterBand(1).YSize
  if n_tiles_exist == 1: print(f"There is a single 'tile' with a height of {tile_size_y_rounded_exist}."); tile_size_y_remainder_exist = 0
  else: print(f"There are {n_tiles_exist} template tiles, the first {n_tiles_exist-1} having a height of {tile_size_y_rounded_exist} pixels, the last {tile_size_y_remainder_exist} pixels.")

In [ ]:
# Large template areas and / or numbers of features may be too much for the available memory.
# This section defines how to split predictions into tiles that can then be merged.
override_n_tiles = True  # Useful if the tile number has already been tested.
n_tiles_override = 1
tile_size_y_rounded_override = int(14910/n_tiles_override)

memory_utilisation = 0.8 # Set to 0.8 to ensure crashes are avoided

assert memory_utilisation > 0 and memory_utilisation <= 1, "Set memory_utilisation to a value between 0 and 1"

# Calculate total size of feature stack
feature_stack_size = template_base_array.size * len(model_scenario_features_dirs)

# Calculate memory and the number of tiles required
total_memory_needed = 64 / 8 * feature_stack_size
print(f'RAM required for each prediction: ~{total_memory_needed/(1024**3):.3f} GB')
print(f'RAM currently available: {psutil.virtual_memory().free / (1024**3):.3f} GB')
n_tiles_temp = int(np.ceil(total_memory_needed / (psutil.virtual_memory().free * memory_utilisation)))

# Calculate template tile size (split on the y axis only)
tile_size_y_rounded = int(np.ceil(template_base_ysize/n_tiles_temp)) # Round the number of y pixels in each tile
tile_size_y_remainder = template_base_ysize%tile_size_y_rounded # Calculate the remainder for the last tile
n_tiles = max(1, len(range(0, template_base_ysize, tile_size_y_rounded))) # Update the number of tiles to include the remainder

if override_n_tiles:
  tile_size_y_rounded = tile_size_y_rounded_override
  tile_size_y_remainder = template_base_ysize%tile_size_y_rounded
  n_tiles = n_tiles_override
  print("n_tiles has been overridden.")

print(f'The prediction template should be divided into {n_tiles} tiles to avoid crashing.')

# Check if tiles need to be changed
change_tiles = True
if override_n_tiles:
  if n_tiles == n_tiles_exist: change_tiles = False
if n_tiles == n_tiles_exist and tile_size_y_rounded == tile_size_y_rounded_exist and tile_size_y_remainder == tile_size_y_remainder_exist:
  change_tiles = False

if change_tiles:
  # Clear all tile directories
  for tile in Path(tile_templates_dir).glob("**/*"):
    if tile.is_file(): tile.unlink()
  for tile in Path(tile_features_dir).glob("**/*"):
    if tile.is_file(): tile.unlink()
  for scenario_stack_dir in Path(tile_feature_stacks_dir).glob("**/*"):
    shutil.rmtree(scenario_stack_dir)
  for tile in Path(tile_prediction_cache_dir).glob("**/*"):
    if tile.is_file(): tile.unlink()

  # Generate new tile templates based on available memory
  tile_number = 1
  for y_start in range(0, template_base_ysize, tile_size_y_rounded):
    if tile_size_y_remainder != 0 and tile_number == n_tiles: tile_size_y = tile_size_y_remainder
    else: tile_size_y = tile_size_y_rounded
    tiling_string = "gdal_translate -of GTIFF -srcwin " + str(0)+ ", " + str(y_start) + ", " + str(template_base_xsize) + ", " + str(tile_size_y) + " " + str(template_base_path) + " " + str(tile_templates_dir) + "/template_tile_" + str(tile_number) + ".tif"
    os.system(tiling_string)
    tile_number += 1
  print("Template tile creation complete.")

else: print("No changes to existing tiles are required.")

In [ ]:
# Create feature tiles.

# Check existing tile parameters
template_tile_list = []
for file in os.listdir(tile_templates_dir):
  if file.endswith('.tif') and file[:13] == 'template_tile':
    template_tile_list.append(file)
n_tiles = len(template_tile_list)

assert n_tiles > 0, "There are no template tiles. Run the template tiles section, even if only one is created."
template_tile_1 = gdal.Open(join(tile_templates_dir, 'template_tile_1.tif'))
tile_size_y_rounded = template_tile_1.GetRasterBand(1).YSize
print(f"There are {n_tiles} template tiles.")

if n_tiles == 1: print("Feature tile creation skipped. Feature stack creation will use the original features.")
else:
  # Progress
  n_features = len(os.listdir(features_dir))
  feature_progress_index, feature_progress_label = 0, widgets.Label(value=f"Feature progress: 0 / {n_features}")
  display(feature_progress_label)
  tile_progress_index, tile_progress_label = 0, widgets.Label(value=f"Tile progress: 0 / {n_tiles}")
  display(tile_progress_label)

  # Loop through each feature in the 6_scenarios features directory
  for feature in os.listdir(features_dir):
    # Create list of tile directories
    feature_dir = join(features_dir, feature)
    feature_array = gdal.Open(feature_dir).ReadAsArray()
    # Split the feature array into chunks based on tile size
    feature_chunks = np.array_split(feature_array, np.arange(tile_size_y_rounded, len(feature_array), tile_size_y_rounded))
    tile_count = 1
    # Loop through tiles and export as .tif
    for tile in range(n_tiles):
      feature_tile_filename = f"{feature[:-4]}_{tile_count}.tif"
      # Check if tile already exists
      feature_tile_exists = False
      for feature_tile in os.listdir(tile_features_dir):
        if feature_tile == feature_tile_filename: feature_tile_exists=True
        # If feature tile does not exist:
      if feature_tile_exists == False:
        template_tile_dir = join(tile_templates_dir, f"template_tile_{tile_count}.tif")
        export_array_as_tif(feature_chunks[tile_count-1], join(tile_features_dir,feature_tile_filename), template_tile_dir, compress = False)
        tile_count += 1
      # Update tile progress
      tile_progress_index += 1
      tile_progress_label.value = f"Tile progress: {tile_progress_index} / {n_tiles}"
    tile_progress_index = 0
    # Update feature progress
    feature_progress_index += 1
    feature_progress_label.value = f"Feature progress: {feature_progress_index} / {n_features}"

# Feature stacks

In [ ]:
# Create feature stack arrays for each scenario

# Collect scenarios with .csv feature lists
scenarios_list = []
for csv in os.listdir(scenarios_model_dir):
  if csv.endswith('.csv'):
    scenarios_list.append(csv[:-4])

# Select scenarios to generate tiled feature stacks
print("scenarios_to_stack = [")
for scenario in sorted(scenarios_list):
  print(f'  "{scenario}",')
print("]")

In [ ]:
scenarios_to_stack = [
  "2018",
  "2019",
  "2020",
  "2021",
  "2021_no_degradation_since_1993",
  "2021_no_disturbance_since_1993",
  "2021_oldgrowth_1",
  "2021_oldgrowth_2",
  "2021_oldgrowth_all_land_1",
  "2021_oldgrowth_all_land_2",
  "2022",
  "2023",
  "2024",
  "2024_no_degradation_since_1996",
  "2024_no_disturbance_since_1996",
  "2024_no_disturbance_since_1997",
  "2024_no_disturbance_since_1998",
  "2024_no_disturbance_since_1999",
  "2024_no_disturbance_since_2000",
  "2024_no_disturbance_since_2001",
  "2024_no_disturbance_since_2002",
  "2024_no_disturbance_since_2003",
  "2024_no_disturbance_since_2004",
  "2024_no_disturbance_since_2005",
  "2024_no_disturbance_since_2006",
  "2024_no_disturbance_since_2007",
  "2024_no_disturbance_since_2008",
  "2024_no_disturbance_since_2009",
  "2024_no_disturbance_since_2010",
  "2024_no_disturbance_since_2011",
  "2024_no_disturbance_since_2012",
  "2024_no_disturbance_since_2013",
  "2024_no_disturbance_since_2014",
  "2024_no_disturbance_since_2015",
  "2024_no_disturbance_since_2016",
  "2024_no_disturbance_since_2017",
  "2024_no_disturbance_since_2018",
  "2024_no_disturbance_since_2019",
  "2024_no_disturbance_since_2020",
  "2024_no_disturbance_since_2021",
  "2024_no_disturbance_since_2022",
  "2024_no_disturbance_since_2023",
  "2024_no_disturbance_since_2024",
  "2024_oldgrowth_1",
  "2024_oldgrowth_2",
  "2024_oldgrowth_all_land_1",
  "2024_oldgrowth_all_land_2",
  "2024_road_mat_daling_deforestation_2023_30m_degradation_buffer",
]

assert set(scenarios_to_stack).issubset(scenarios_list), "Not all selected scenarios exist."

# Check existing tile parameters
template_tile_list = []
for file in os.listdir(tile_templates_dir):
  if file.endswith('.tif') and file[:13] == 'template_tile':
    template_tile_list.append(file)
n_tiles = len(template_tile_list)
assert n_tiles > 0, "There are no template tiles. Run the template tiles section, even if only one is created."
print(f"There are {n_tiles} template tiles.")
# Set the number of stacks to the number of tiles
n_stacks = n_tiles

# Progress
scenario_progress_index, scenario_progress_label = 0, widgets.Label(value=f"Scenario progress: 0 / {len(scenarios_to_stack)}")
display(scenario_progress_label)
stack_progress_index, stack_progress_label = 0, widgets.Label(value=f"Tiled feature stack progress: 0 / {n_tiles}")
display(stack_progress_label)

# Loop through each scenario
for scenario in scenarios_to_stack:
    # Define directory and features
    scenario_feature_stacks_dir = join(tile_feature_stacks_dir, scenario)
    makedirs(scenario_feature_stacks_dir, exist_ok=True)
    scenario_features_csv = join(scenarios_model_dir,f"{scenario}.csv")
    scenario_features = pd.Series.tolist(pd.read_csv(scenario_features_csv).iloc[:,0])
    # Create a tile count to match the feature stack
    tile_count = 1
    for tile in range(n_stacks):
      scenario_stack_filename = f"feature_stack_{scenario}_{tile_count}.npy"
      # Check if feature stack already exists
      feature_stack_exists = False
      for feature_stack in os.listdir(scenario_feature_stacks_dir):
        if feature_stack == scenario_stack_filename: feature_stack_exists=True
      # If scenario prediction tile does not exist:
      if feature_stack_exists == False:
        scenario_tile_stack_dir = join(scenario_feature_stacks_dir, scenario_stack_filename)
        # Create feature chunks (arrays) from tiles
        if n_stacks == 1: feature_tiles_dirs = [f"{features_dir}/{feature}.tif" for feature in scenario_features]
        else: feature_tiles_dirs = [f"{tile_features_dir}/{feature}_{tile_count}.tif" for feature in scenario_features]
        feature_array_chunks = []
        for feature in feature_tiles_dirs:
          # Covariate raster will exist and should be ignored if prediction stage has already been attempted
          if feature.split('/')[-1].split('.')[0] not in covariates and feature.split('/')[-1].split('.')[0] not in [f"{cov}_{tile_count}" for cov in covariates]:
            feature_array_chunk = gdal.Open(feature).ReadAsArray()
            feature_array_chunks.append(feature_array_chunk)
        # Create a feature stack from chunks
        feature_stack = np.dstack(feature_array_chunks)
        feature_array_chunks = None # Flush chunks
        stack_height, stack_width, stack_n_features = feature_stack.shape
        # Convert feature stack to 2D numpy array with features as columns
        feature_stack_reshaped = feature_stack.reshape(stack_height * stack_width, stack_n_features)
        feature_stack = None # Flush stack
        # Save as a numpy file
        np.save(scenario_tile_stack_dir, feature_stack_reshaped)

        # Force Drive sync and verify
        subprocess.run(['sync'], check=True)
        last_size = -1
        for attempt in range(10):
            time.sleep(5)
            if os.path.exists(scenario_tile_stack_dir):
                current_size = os.path.getsize(scenario_tile_stack_dir)
                if current_size == last_size and current_size > 0:
                    try:
                        np.load(scenario_tile_stack_dir)
                        break
                    except: pass
                last_size = current_size
        else: raise RuntimeError(f"Drive sync failed: {scenario_tile_stack_dir}")

        feature_stack_reshaped = None # Flush reshaped stack
      # Update progress
      tile_count += 1
      stack_progress_index += 1
      stack_progress_label.value = f"Tiled feature stack progress: {stack_progress_index} / {n_stacks}"
    # Reset tile progress
    stack_progress_index = 0
    scenario_progress_index += 1
    scenario_progress_label.value = f"Scenario progress: {scenario_progress_index} / {len(scenarios_to_stack)}"
print("\nFeature stacks complete.")

# Predict scenarios

In [ ]:
# This is for testing models and scenarios, or making predictions where no
# uncertainty metric for the variate (e.g. standard error or stdev) is available.
# If these are available, proceed to 7_uncertainty.ipynb.

# Check existing tile parameters
template_tile_list = []
for file in os.listdir(tile_templates_dir):
  if file.endswith('.tif') and file[:13] == 'template_tile':
    template_tile_list.append(file)
n_tiles = len(template_tile_list)
assert n_tiles > 0, "# There are no template tiles. Run the template tiles section, even if only one is created."
if n_tiles == 1: print(f"# There is 1 template tile.\n")
if n_tiles > 1: print(f"# There are {n_tiles} template tiles.\n")

# Collect available scenarios from the feature stack tiles directory
scenario_stacks_list = []
for scenario in os.listdir(tile_feature_stacks_dir):
    if len(os.listdir(join(tile_feature_stacks_dir, scenario))) == n_tiles:
        scenario_stacks_list.append(scenario)

print("# Note: If you end a runtime after the creation of many large feature stacks,")
print("# it will time for the notebook to recognise their existence again due to")
print("# Google Drive latency issues. If the stacks do not appear here after some time,")
print("# run the feature stack section again until they do.\n")
# Select scenarios to predict
print("scenarios_to_predict = [")
for scenario in sorted(scenario_stacks_list):
  print(f'  "{scenario}",')
print("]")

In [ ]:
# There is 1 template tile.

# Note: If you end a runtime after the creation of many large feature stacks,
# it will time for the notebook to recognise their existence again due to
# Google Drive latency issues. If the stacks do not appear here after some time,
# run the feature stack section again until they do.

scenarios_to_predict = [
  "2018",
  "2019",
  "2020",
  "2021",
  "2021_no_degradation_since_1993",
  "2021_no_disturbance_since_1993",
  "2021_oldgrowth_1",
  "2021_oldgrowth_2",
  "2021_oldgrowth_all_land_1",
  "2021_oldgrowth_all_land_2",
  "2022",
  "2023",
  "2024",
  "2024_no_degradation_since_1996",
  "2024_no_disturbance_since_1996",
  "2024_no_disturbance_since_1997",
  "2024_no_disturbance_since_1998",
  "2024_no_disturbance_since_1999",
  "2024_no_disturbance_since_2000",
  "2024_no_disturbance_since_2001",
  "2024_no_disturbance_since_2002",
  "2024_no_disturbance_since_2003",
  "2024_no_disturbance_since_2004",
  "2024_no_disturbance_since_2005",
  "2024_no_disturbance_since_2006",
  "2024_no_disturbance_since_2007",
  "2024_no_disturbance_since_2008",
  "2024_no_disturbance_since_2009",
  "2024_no_disturbance_since_2010",
  "2024_no_disturbance_since_2011",
  "2024_no_disturbance_since_2012",
  "2024_no_disturbance_since_2013",
  "2024_no_disturbance_since_2014",
  "2024_no_disturbance_since_2015",
  "2024_no_disturbance_since_2016",
  "2024_no_disturbance_since_2017",
  "2024_no_disturbance_since_2018",
  "2024_no_disturbance_since_2019",
  "2024_no_disturbance_since_2020",
  "2024_no_disturbance_since_2021",
  "2024_no_disturbance_since_2022",
  "2024_no_disturbance_since_2023",
  "2024_no_disturbance_since_2024",
  "2024_oldgrowth_1",
  "2024_oldgrowth_2",
  "2024_oldgrowth_all_land_1",
  "2024_oldgrowth_all_land_2",
  "2024_road_mat_daling_deforestation_2023_30m_degradation_buffer",
]

In [ ]:
# GEDI covariates need to be changed to a set value for all predictions
add_covariates = True # Adds a selected covariate value as the feature
sensitivity_value = 0.99
# Higher sensitivity indicative of GEDI footprint 'quality'
# However it may also overestimate vegetation metrics like AGBD.
# If predictions appear to have a positive bias, lower this and run again.
beam_value = 5
# 5 is the first of the full beams, which appears to have the least bias on vegetation metrics.
# Cover beams 1 - 4 underestimate. Full beams 7 - 8 overestimate. 5 - 6 tend give average values.

# Detect GPU availability and set predictor type
try:
    test_array = cupy.array([1, 2, 3]) # Test if GPU is actually accessible
    del test_array
    predictor_type = 'gpu_predictor'
    gpu_id, use_gpu = 0, True
    print("GPU detected and accessible - using GPU to load the feature stack and to predict.")
    @contextmanager
    def gpu_memory_context():
        try: yield
        finally:
            cupy.cuda.Device().synchronize()
            gc.collect()
            cupy.get_default_memory_pool().free_all_blocks()
            cupy.get_default_pinned_memory_pool().free_all_blocks()
except:
    predictor_type = 'cpu_predictor'
    gpu_id = -1  # -1 means CPU
    use_gpu = False
    print("GPU not accessible - using CPU prediction")
xgb.set_config(verbosity=0, use_rmm=use_gpu)

# Load model description to get proper parameters
model_description_path = selected_model_json.replace('model.json', 'model_description.json')
with open(model_description_path) as f: model_desc = json.load(f)
model_params = eval(model_desc["hyperparameters"])
# Detect classification and multiclass from model parameters
classification = 'objective' in model_params and ('logistic' in model_params['objective'] or 'softprob' in model_params['objective'])
multiclass = 'num_class' in model_params and model_params['num_class'] > 2
print(f"Model type detected: {'Classification' if classification else 'Regression'}")
if classification and multiclass: print(f"Multiclass with {model_params['num_class']} classes")

# Load model and create predictor with proper parameters
if classification:
    # Create XGBClassifier with original parameters
    XGBPredictor = xgb.XGBClassifier(**model_params)
    XGBPredictor.load_model(selected_model_json)
    XGBPredictor.set_params(predictor=predictor_type)
    if use_gpu: XGBPredictor.set_params(gpu_id=gpu_id)
else:
    # For regression, use booster approach
    booster = xgb.Booster()
    booster.load_model(selected_model_json)
    booster.set_param('predictor', predictor_type)
    if use_gpu: booster.set_param('gpu_id', gpu_id)
    XGBPredictor = xgb.XGBRegressor(predictor=predictor_type, gpu_id=gpu_id if use_gpu else -1)
    XGBPredictor._Booster = booster

# Check existing tile parameters
template_tile_list = [file for file in os.listdir(tile_templates_dir)
                     if file.endswith('.tif') and file[:13] == 'template_tile']
n_tiles = len(template_tile_list)
assert n_tiles > 0, "There are no template tiles. Run the template tiles section, even if only one is created."
template_tile = gdal.Open(join(tile_templates_dir,'template_tile_1.tif'))
template_tile_x = template_tile.GetRasterBand(1).XSize
print(f"There are {n_tiles} template tiles.")

# Progress tracking
scenario_progress_index = 0
scenario_progress_label = widgets.Label(f"Scenario progress: {scenario_progress_index}/{len(scenarios_to_predict)}")
display(scenario_progress_label)
tile_progress_index, tile_progress_label = 0, widgets.Label(value=f"Tile progress: 0 / {n_tiles}")
display(tile_progress_label)

# Loop through each scenario
for scenario in scenarios_to_predict:
  scenario_prediction_unmasked_filename = f"{scenario}__{selected_model}_unmasked.tif"
  scenario_prediction_unmasked_exists = scenario_prediction_unmasked_filename in os.listdir(scenario_predictions_unmasked_dir)
  if not scenario_prediction_unmasked_exists:
    scenario_feature_stack_dir = join(tile_feature_stacks_dir, scenario)
    n_stacks = len(os.listdir(scenario_feature_stack_dir))
    tile_cache_scenario_dir = join(tile_prediction_cache_dir, scenario_prediction_unmasked_filename[:-4])
    makedirs(tile_cache_scenario_dir, exist_ok=True)
    for stack in range(1, n_stacks+1):
      scenario_tile_filename = f"scenario_tile_{stack}.tif"
      scenario_tile_exists = scenario_tile_filename in os.listdir(tile_cache_scenario_dir)
      if not scenario_tile_exists:
        # Clear GPU memory before new stack
        if use_gpu:
          with gpu_memory_context(): pass
        # Load template tile parameters
        template_tile_dir = join(tile_templates_dir, f"template_tile_{stack}.tif")
        template_tile = gdal.Open(template_tile_dir)
        template_tile_y = template_tile.GetRasterBand(1).YSize
        template_tile_x = template_tile.GetRasterBand(1).XSize
        # Load stack to GPU or CPU
        stack_filename = f"feature_stack_{scenario}_{stack}.npy"
        stack_path = join(scenario_feature_stack_dir, stack_filename)
        if use_gpu:
          try:
            feature_stack = cupy.load(stack_path)
            if add_covariates: feature_stack = cupy.hstack((feature_stack,
                  cupy.full((feature_stack.shape[0], 1), beam_value, dtype=int),
                  cupy.full((feature_stack.shape[0], 1), sensitivity_value, dtype=float)))
          except Exception as e:
            if "Memory allocation error" in str(e) or "Out of memory" in str(e):
              print("GPU memory insufficient for feature stack, loading stack with CPU.")
              feature_stack = np.load(stack_path)
              if add_covariates: feature_stack = np.hstack((feature_stack,
                    np.full((feature_stack.shape[0], 1), beam_value, dtype=int),
                    np.full((feature_stack.shape[0], 1), sensitivity_value, dtype=float)))
              with gpu_memory_context(): pass
            else: raise
        else:
          feature_stack = np.load(stack_path)
          if add_covariates: feature_stack = np.hstack((feature_stack,
                            np.full((feature_stack.shape[0], 1), beam_value, dtype=int),
                            np.full((feature_stack.shape[0], 1), sensitivity_value, dtype=float)))
        # Predict - terminate runtime if GPU prediction fails
        try:
            prediction = XGBPredictor.predict(feature_stack)
            # Process predictions based on model type
            if classification:
                # Check if prediction is 2D (probabilities) and convert to class labels
                if prediction.ndim > 1 and prediction.shape[1] > 1: prediction = np.argmax(prediction, axis=1)
                elif prediction.ndim == 1: # For binary classification with single probability values
                  prediction = np.round(prediction).astype(int)
                  # Ensure prediction is integer type for classification
                  prediction = prediction.astype(int)
        except Exception as e:
          if "Memory allocation error" in str(e) or "Out of memory" in str(e):
            print("GPU memory insufficient for prediction. Terminating runtime to save compute units, restart with TPU.")
            runtime.unassign()
          else: raise
        feature_stack = None # Flush feature stack
        prediction_tile = prediction.reshape((template_tile_y, template_tile_x))
        prediction = None # Flush prediction
        # Export prediction array as .tif
        export_array_as_tif(prediction_tile, join(tile_cache_scenario_dir, scenario_tile_filename),
                          template = template_tile_dir, compress = False)
        prediction_tile = None # Flush prediction tile
      tile_progress_index += 1
      tile_progress_label.value = f"Tile progress: {tile_progress_index} / {n_stacks}"
    # Prepare empty array for merging tiles
    prediction_array = np.empty((0,template_tile_x))
    # Read each tile .tif as an array, stack, then export as a .tif
    for subdir in os.listdir(tile_cache_scenario_dir):
      if subdir.endswith('.tif'):
        tile_dir = join(tile_cache_scenario_dir, subdir)
        prediction_array = np.vstack((prediction_array, gdal.Open(tile_dir).ReadAsArray()))
    # Define scenario template
    scenario_template = join(features_dir, os.listdir(features_dir)[0])
    scenario_prediction_unmasked_dir = join(scenario_predictions_unmasked_dir, f"{scenario_prediction_unmasked_filename}")
    export_array_as_tif(prediction_array, scenario_prediction_unmasked_dir, template = scenario_template, compress = True)
    # Delete scenario tile cache directory
    shutil.rmtree(tile_cache_scenario_dir)
  # Reset tile progress
  tile_progress_index = 0
  # Update scenario progress
  scenario_progress_index += 1
  scenario_progress_label.value = f"Scenario progress: {scenario_progress_index}/{len(scenarios_to_predict)}"
print("\nScenario predictions complete.")

# Mask scenario predictions



In [ ]:
# Use polygons for masking, only areas inside the polygons will be included

# Exclude existing polygons from search
polygons_to_exclude = ['template.gpkg', 'project_area_buffered_bbox.gpkg']

print("mask_polygons = [")
for polygon in os.listdir(polygons_dir):
  if polygon not in polygons_to_exclude:
    if 'inverse' not in polygon:
      print(f"  '{polygon[:-5]}',")
print("]")

In [ ]:
mask_polygons = [
  # 'project_area',
  'gedi_area',
  # 'peninsular_malaysia',
  # 'pa_taman_krau',
  # 'pa_ais',
]

# Create an inverse project area path for masking
template_polygon_path = join(polygons_dir, "template.gpkg")
for polygon in mask_polygons:
  inverse_polygon_path = join(polygons_dir, f"{polygon}_inverse.gpkg")
  if not exists(inverse_polygon_path):
    polygon_path = join(polygons_dir, f"{polygon}.gpkg")
    template_polygon = gpd.read_file(template_polygon_path)
    polygon_read = gpd.read_file(polygon_path)
    polygon_crs = polygon_read.crs.to_epsg()
    inverse_polygon = template_polygon['geometry'].difference(polygon_read['geometry']).iloc[0]
    inverse_polygon_gdf = gpd.GeoDataFrame({'geometry': [inverse_polygon]}, crs=f"EPSG:{polygon_crs}")
    inverse_polygon_gdf.to_file(inverse_polygon_path, driver="GPKG")
    print(f"An inverse masking polygon for {polygon} has been created in {polygons_dir}.")
  else: print(f"An inverse masking polygon for {polygon} already exists.")


# If only [oldgrowth_scneario]_1 exists, all disturbance from all disturbance features is removed
# If [oldgrowth_scneario]_1 and [oldgrowth_scneario]_2 exist,
# [oldgrowth_scneario]_1 uses a land-use proxy for pre-Landsat undisturbed forest
# [oldgrowth_scneario]_2 simply removes all disturbance from all disturbance features
# The final masked [scenario]_oldgrowth chooses the maximum pixel values from comparing each.

# Merge oldgrowth versions by taking maximum values
oldgrowth_version1_files = [f for f in os.listdir(scenario_predictions_unmasked_dir)
                           if f.split('__')[0].endswith('_1') and f.endswith('_unmasked.tif')]
for v1_file in oldgrowth_version1_files:
  base_name = v1_file.split('__')[0][:-1] + '2'  # Replace '1' with '2'
  rest_of_name = '__' + v1_file.split('__')[1]
  v2_file = f"{base_name}{rest_of_name}"
  merged_file = v1_file.replace('_1__', '__')
  merged_path = join(scenario_predictions_unmasked_dir, merged_file)
  # Skip if merged file already exists
  if exists(merged_path): continue
  # Check if version 2 exists
  if exists(join(scenario_predictions_unmasked_dir, v2_file)):
    print(f"Merging oldgrowth versions for {v1_file.split('__')[0]}...")
    # Load both arrays and take maximum values
    array1 = gdal.Open(join(scenario_predictions_unmasked_dir, v1_file)).ReadAsArray()
    array2 = gdal.Open(join(scenario_predictions_unmasked_dir, v2_file)).ReadAsArray()
    merged_array = np.maximum(array1, array2)
    # Save merged file
    export_array_as_tif(merged_array, merged_path, compress=True)
    print(f"Merged version exported to {merged_file}")
  else:
    # Use version 1 if version 2 doesn't exist
    shutil.copy2(join(scenario_predictions_unmasked_dir, v1_file), merged_path)
    print(f"Version 2 not found, copied version 1 to {merged_file}")

# Collect unmasked predictions, properly skipping oldgrowth version files
unmasked_predictions = []
for scenario_prediction in os.listdir(scenario_predictions_unmasked_dir):
 if '_1__' not in scenario_prediction and '_2__' not in scenario_prediction:
    unmasked_predictions.append(scenario_prediction)

# Determine last feature year for masking future scenarios
final_feature_years = []
for final_feature in os.listdir(feature_final_dir):
  if final_feature.endswith('.tif') and final_feature[-9] == '_':
    try: final_feature_years.append(int(final_feature[-8:-4]))
    except: continue
last_feature_year = max(final_feature_years)

# Binary progress
masking_progress_index = 0
masking_progress_label = widgets.Label(f"Masking progress: {masking_progress_index}/{len(unmasked_predictions)}")
display(masking_progress_label)

# Mask scenarios with the relevatant mask
for scenario_prediction in unmasked_predictions: # Loop through each unmasked scenario
  scenario_masked_filename = f"{scenario_prediction[:-13]}.tif"
  scenario_masked_dir = join(scenario_predictions_dir, scenario_masked_filename)
  scenario_year = int(scenario_prediction[:4])
  if not exists(scenario_masked_dir):
    mask_exists = False
    for mask in os.listdir(masks_dir):
      mask_year = int(mask[12:16])

      # Match area-based deforestation scenarios
      if 'deforestation' in scenario_prediction and 'deforestation' in mask:
          mask_middle = mask[12:-4]  # Remove "mask_forest_" and ".tif"
          if scenario_prediction.startswith(mask_middle):
              selected_mask_filename = mask
              selected_mask_dir = join(masks_dir, selected_mask_filename)
              mask_exists = True

      elif 'deforestation' not in scenario_prediction and 'deforestation' not in mask:

        # Match all oldgrowth scenarios
        if 'oldgrowth_all_land' in mask or 'oldgrowth_all_land' in scenario_prediction:
          if f'{scenario_year}_oldgrowth_all_land' in mask and f'{scenario_year}_oldgrowth_all_land' in scenario_prediction:
            selected_mask_filename = mask
            selected_mask_dir = join(masks_dir, selected_mask_filename)
            mask_exists = True

        elif 'oldgrowth_all_land' not in mask or 'oldgrowth_all_land' not in scenario_prediction:

          if 'disturbance_since' in scenario_prediction:
              disturbance_scenario = scenario_masked_filename.split('__')[0]
              disturbance_since_year = int(disturbance_scenario[-4:])-1
              if disturbance_since_year == mask_year:
                selected_mask_filename = mask
                selected_mask_dir = join(masks_dir, selected_mask_filename)
                mask_exists = True

          else: # Match all other historic scenarios
            if scenario_year == mask_year:
              selected_mask_filename = mask
              selected_mask_dir = join(masks_dir, selected_mask_filename)
              mask_exists = True

            else: # Match future scenarios with most recent forest mask
              if scenario_year > last_feature_year and last_feature_year == mask_year:
                selected_mask_filename = mask
                selected_mask_dir = join(masks_dir, selected_mask_filename)
                mask_exists = True

    if mask_exists == False: print(f"A suitable mask for {scenario_prediction} does not exist.\n")
    else: # Mask the scenario prediction
      print(f"Masking {scenario_prediction} with {selected_mask_filename}...")
      mask_array = gdal.Open(selected_mask_dir).ReadAsArray()
      scenario_prediction_unmasked_dir = join(scenario_predictions_unmasked_dir, scenario_prediction)
      scenario_prediction_array = gdal.Open(scenario_prediction_unmasked_dir).ReadAsArray()
      # Mask where the mask array is not 1
      scenario_masked_array = np.where(mask_array != 1, nodatavalue, scenario_prediction_array)
      export_array_as_tif(scenario_masked_array, scenario_masked_dir, compress = True)
      if len(mask_polygons) > 0:
        for polygon_mask in mask_polygons:
          inverse_gedi_area_path = join(polygons_dir, f"{polygon_mask}_inverse.gpkg")
          print(f"Masking {scenario_prediction} with {polygon_mask}...")
          burn_polygon_to_raster(scenario_masked_dir, inverse_gedi_area_path, fixed_value=nodatavalue, all_touched=False)
        # Recompress the prediction after burning the polygon masks
        scenario_masked_array_2 = gdal.Open(scenario_masked_dir).ReadAsArray()
        export_array_as_tif(scenario_masked_array_2, scenario_masked_dir, compress = True)
      print(f"{scenario_masked_filename} exported.")
  # Update masking progress
  masking_progress_index += 1
  masking_progress_label.value = f"Masking progress: {masking_progress_index}/{len(unmasked_predictions)}"

# Disconnect runtime

In [ ]:
# Useful for stopping background execution
runtime.unassign()